# Data Analytics Spring 2024 &mdash; Exercises 2

### XXXXX XXXXX

Note from Harri: These are old, but that shouldn't matter much. The role of the model solutions is to provide peer review guidelines by showing what kind of a solution deserves five points.

The problem 1 profiles are in UK format (this year we have US format).

## Problem 1. Profiles

The file `private/exrc_02/data/XXXXX_prob01_profiles.csv` contains some user profiles.
Read the csv into a pandas DataFrame and reorganize it as follows:

a) Separate the name and address columns so that there are separate columns for
- first name
- last name
- street address
- state
- postal code.

Keep also the ssn, username, sex, mail and birthdate columns. Drop all the other columns.

b) Print all entries where the last name begins with the letter J, sorted by:
- sex (ladies first)
- state (alphabetically)
- age (youngest first).

I.e. print the entries three times, in these three different ways. 

In [ ]:
# Imports for this and all subsequent problems
import pandas as pd

In [ ]:
# Read in data
df = pd.read_csv('data/XXXXX_prob01_profiles.csv')
print(f"Dataframe shape: {df.shape}")
df.head()

In [ ]:
#----------------------------------------------------------------------
# Part a) separate name and address columns into first name, last name
# street address, state, postal code
#----------------------------------------------------------------------

# ---------------- SANITY CHECK ----------------------------
# There are titles in front of the names. But not always. These need to be accounted for.
# Sanity checks to understand what to do with the data:
# - Split the name into parts (separated by space)
split_names = df.name.map(lambda x: x.split(' '))

# - What is the largest number of splits? Would expect 3 (title, first name, last name)
print(f"Maximum number of splits: {split_names.map(lambda x: len(x)).max()}")
# Result: 3. 

# So, let's check what values are in the first splits of 3-part names. 
# Because we might still have cases with FirstName1 FirstName2 Surname
# (or like the Spanish, FirstName MomsSurname DadsSurname)

unique_first_parts = {}

for x in split_names:
    if len(x) > 2:
        unique_first_parts[x[0]] = ''

print(f"Unique values as first words in three-part names: {', '.join(unique_first_parts.keys())}")
# Result: Mr, Dr, Miss, Mrs, Ms

# So: if the name is composed of three parts, the first part is always the title. 
# This lets us assume that we can just take the last two parts of the name field, and
# they will be the First Name and the Last Name. 
# ----------------- SANITY CHECK ENDS ------------------------

In [ ]:
# Create first_name and last_name columns in the data frame
df['first_name'] = df.name.map(lambda x: x.split(' ')[-2])
df['last_name'] = df.name.map(lambda x: x.split(' ')[-1])

# Example data
print(df[['first_name', 'last_name']].head())

In [ ]:
# ------------------ MORE SANITY CHECKS ---------------------------
# What about the address?
split_addresses = df.address.map(lambda x: x.split('\n'))

# What is the maximum number of address splits?
print(f"Maximum number of address splits: {split_addresses.map(lambda x: len(x)).max()}\n")
# Result: 4

# At this point I'm making an assumption that the first part may be a specifier
# (like a flat number), second part is street name, third part city and last one postal code.
# -------------------SANITY CHECK ENDS ----------------------------

df['street_address'] = df.address.map(lambda x: x.split('\n')[-3])
df['state'] = df.address.map(lambda x: x.split('\n')[-2])
df['postal_code'] = df.address.map(lambda x: x.split('\n')[-1])

# Sample data
print(df[['street_address', 'state', 'postal_code']].head())

# Instruction: Keep also the ssn, username, sex, mail and birthdate columns. Drop all the other columns.
columns_to_keep = ['first_name', 'last_name', 'street_address', 'state', 'postal_code', 'ssn', 'username', 'sex', 'mail', 'birthdate']
df = df[columns_to_keep]

# Results
df.head()

In [ ]:
# ----------------------------------------------------------------------------
# b) Print all entries where the last name begins with the letter J, sorted by:
#
# sex (ladies first)
# state (alphabetically)
# age (youngest first).
#-----------------------------------------------------------------------------

# Filtering all whose last name starts with J
all_Js = df[df['last_name'].str.startswith('J')]

# Ladies first
all_Js.sort_values(by='sex')

In [ ]:
# State (alphabetically)
all_Js.sort_values(by='state')

In [ ]:
# Age (youngest first)
all_Js.sort_values(by='birthdate', ascending=False)

## Problem 2. Weather (part 1/2)

The file `private/exrc_02/data/XXXXX_prob02_weather.csv` contains hourly weather observations from Helsinki during one month, downloaded from [fmi.fi](https://en.ilmatieteenlaitos.fi/open-data-manual-fmi-wfs-services) (not recommended).

First, please do some data cleaning and reorganizing as you find suitable. Then, please answer the following questions:

a) How many percentages of the `(tmax+tmin)/2` observations are at most one standard deviation away from the total average of `(tmax+tmin)/2`?

b) Find the top-5 timestamps for the difference between `tmax` and `tmin`, i.e. for `tmax-tmin`. For the found rows, print out the following information: timestamp, max temperature, min temperature and temperature difference.


In [ ]:
# Read in file
df = pd.read_csv('data/XXXXX_prob02_weather.csv')

# Get some info out of the DataFrame
print(df.shape)
print(df.describe())
df.head()

In [ ]:
# -----------------------------------------------------------------------------
# "First, please do some data cleaning and reorganizing as you find suitable."
# -----------------------------------------------------------------------------
# Well. There are at least three major things to tackle outright. 
# 1) Setting the index to a proper value (the datetime would be a good candidate)
# 2) ParameterName and ParameterValue columns contain key-value pairs for specific timestamps.
#    These should probably be split into their own columns. One row of data per timestamp.
# 3) First column can be dropped (only contains an index number)

# Turns out that all of these can be done in a single line with pivot_table:
df = df.pivot_table(index='Time', columns='ParameterName', values='ParameterValue')
df = df.rename_axis(index='Time')
df

In [ ]:
# Looking at the data, one more tweak needs to be made. TG_PT12H_min and the
# rest of the data points are from the same day, but from six hours apart. Because
# of this, they reside in different rows. For the purpose of this exercise, having
# all the values in a single row would be easier. So, let's merge the timestamps.

# Drop the hour information from timestamps
df.set_index(pd.to_datetime(df.index).date, inplace=True)
df = df.rename_axis(index='Time')
df.head()

In [ ]:
# This line groups the timestamps in the dataframe together, and selects the first 
# non-null value (x.dropna() removes all nulls, iloc[0] selects the first one from
# the values that remain. The if section ensures that a "None" is returned as the
# value of there were no values to begin with (like for TG_PT12H_min, days 16.11.-->)
df = df.groupby(df.index).agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else None)
df.head()

In [ ]:
# Info regarding the data at this stage
print(df.shape)
print(df.describe())
df.head()

In [ ]:
# The -1.0 values in rrday and snow probably correlate to a "did not snow/rain" value.
# I did not change these values, as I was not sure what the significance of -1.0 was.
# I also didn't change the column headers to a more human-readable format, as I would
# need to understand the data better for this. I can only assume what the headers mean. 

In [ ]:
# ---------------------------------------------------------------------------
# a) How many percentages of the (tmax+tmin)/2 observations are at most one 
#    standard deviation away from the total average of (tmax+tmin)/2?
# ---------------------------------------------------------------------------

# Let's add a column with value for (tmax+tmin)/2, which would be tavg (average)
df ['tavg'] = (df.tmax + df.tmin) / 2

df.head()

In [ ]:
# Calculations for the result:
# Get standard deviation and mean
tavg_std = df.tavg.std()
tavg_mean = df.tavg.mean()

# Create filter mask to check whether tavg-value is within one standard deviation
filter_mask = (df.tavg >= (tavg_mean - tavg_std)) & (df.tavg <= (tavg_mean + tavg_std))

# Filter dataframe
filtered = df[filter_mask]

# Calculate percentage that passed the filter
print(f"{round(len(filtered)/len(df) * 100, 2)}% of (tmax+tmin)/2 values were within one standard deviation of the mean.\n")

# Sanity check: 
# Get min and max of those within one std:
print("Sanity:")
print(f"Minimum that made it: {filtered.tavg.min()}, maximum that made it: {filtered.tavg.max()}")
print(f"Distance between max and min: {round(filtered.tavg.max() - filtered.tavg.min(), 2)}")
print(f"Two standard deviations: {round(tavg_std * 2, 2)}")

# Check if values that got left out were much smaller or larger than those included
print(f"Minimum that got left out: {df[~filter_mask].tavg.min()}, maximum that got left out: {df[~filter_mask].tavg.max()}")


In [ ]:
# ---------------------------------------------------------------------------
# b) Find the top-5 timestamps for the difference between tmax and tmin, i.e. 
#    for tmax-tmin. For the found rows, print out the following information: 
#    timestamp, max temperature, min temperature and temperature difference.
# ---------------------------------------------------------------------------

# Again, it's easiest to just create a new column for this. Calling it 'tdiff'.
df['tdiff'] = df.tmax - df.tmin
df.head()

In [ ]:
# Sort values by tdiff
df = df.sort_values(by='tdiff', ascending=False)

# Print out tmax, tmin and tdiff for top five temperature differences
df.iloc[:5][['tmax', 'tmin', 'tdiff']]

## Problem 3. Premier League Table
The file `private/exrc_02/data/XXXXX_prob03_epl.csv` has some English Premier League results, downloaded using [this api](https://github.com/miquel-vv/football_data_api).

Using the full data, generate a league table which has the team name as the index and columns as follows (a win gives 3 points, a draw gives 1 point, and a loss gives 0 points):
* games played
* wins
* draws
* defeats
* goals for - goals against
* points


Sort it with points (most points win). If points are equal, then sorted by
* goal difference (goals for - goals against)
* goals for


The expected result should look something like this (not the same data though):
```
                games  wins  draws  defeats   goals  points
Man City           38    32      4        2  106-27     100
Man United         38    25      6        7   68-28      81
Tottenham          38    23      8        7   74-36      77
Liverpool          38    21     12        5   84-38      75
...
```

In [ ]:
# Read in data.
df = pd.read_csv('data/XXXXX_prob03_epl.csv')

# Basic info about data
print(df.shape)
print(df.describe())
df.head()

In [ ]:
# First things first, let's get the values in fullTime into their own columns.

# Grabbing some tools:
import ast

# Turn the dictionary string into an actual dictionary
goals = df.fullTime.apply(ast.literal_eval)

# Create a dataframe from the dictionary and add to existing dataframe
df[['homeGoals', 'awayGoals']] = pd.DataFrame(goals.tolist())

# Drop the original fullTime column
df.drop('fullTime', axis='columns', inplace=True)

# Set homeGoals and awayGoals to integers instead of floats
df['homeGoals'] = df['homeGoals'].fillna(0).astype(int)
df['awayGoals'] = df['awayGoals'].fillna(0).astype(int)

df.head()

In [ ]:
# Next step is to split home and away teams to get their statistics. Each game will
# produce two rows of data: one for home games, one for away games, looking at the
# statistics from the point of the team. 

#  Home game teams first:
home_games = df[['homeTeam', 'homeGoals', 'awayGoals']].copy()
home_games.columns = ['team', 'goals_for', 'goals_against']
home_games

In [ ]:
# Same for away teams
away_games = df[['awayTeam', 'awayGoals', 'homeGoals']].copy()
away_games.columns = ['team', 'goals_for', 'goals_against']
away_games

In [ ]:
# Merge home_games and away_games
league_table = pd.concat([home_games, away_games])
league_table

In [ ]:
# Calculation of the data the problem asks for: 
# games played
# wins
# draws
# defeats
# goals for - goals against
# points

# Creating new columns for game-by-game statistics, which can be
# summed up later to get the answer to Problem 3.
league_table['games played'] = 1
league_table['wins'] = league_table.goals_for > league_table.goals_against
league_table['draws'] = league_table.goals_for == league_table.goals_against
league_table['defeats'] = league_table.goals_for < league_table.goals_against
league_table['points'] = 3 * league_table.wins + league_table.draws

league_table

In [ ]:
# Group game data by team and sum up the statistics
league_table = league_table.groupby('team').agg('sum')

# Add a goals-column
league_table['goals'] = league_table.goals_for.astype(str) + '-' + league_table.goals_against.astype(str)

league_table

In [ ]:
# Sort and format table to provide the answer to Problem 3:
# Helper column for sorting by goal difference
league_table['goal_diff'] = league_table.goals_for - league_table.goals_against

# Sort table
league_table = league_table.sort_values(by=['points', 'goal_diff', 'goals_for'], ascending=False)

# Keep only those columns that were requested in the order requested
league_table = league_table[['games played', 'wins', 'draws', 'defeats', 'goals', 'points']]
league_table

## Problem 4. Weather (part 2/2)

The file `private/exrc_02/data/XXXXX_prob04_weather.txt` 
has some (old) weather data from Jyväskylä 1959-2021, again downloaded from [fmi.fi](https://en.ilmatieteenlaitos.fi/open-data-manual-fmi-wfs-services) (not recommended).

Calculate the "snow sum" (not an official meteorological term) for each winter by adding the snow depths for each day of that winter. Start from winter 1959-60 and end to 2019-20 since 1958-59 and 2020-21 are only partial.

Notes:
* You need to define "winter" by yourself.
* FMI uses -1 as snow depth when "there is absolutely no snow at all". We don't want to reduce snow sum in that case, so replace -1 with 0.
* For missing data, assume that the snow depth has been the same as during the previous day. (Fill any `NaN`s with the previous valid value.)

Then produce a DataFrame that has the winter as the index (in form "1959-1960") and columns:
* snow sum
* snow sum rank among winters so that largest = 1
* count of days where snow depth has been positive
* max snow depth of the winter.


The three first and the three last rows should look something like:
```
           Snow sum  rank  count  max
Winter                               
1959-1960      5593    18    169   65
1960-1961      5082    28    162   60
1961-1962      6644    12    156   78

...

2017-2018      6882     8    161   81
2018-2019      4030    42    150   54
2019-2020      1432    59    112   30
```

In [ ]:
# Read in data
df = pd.read_csv('data/XXXXX_prob04_weather.csv')

# Get info about the data structure
print(df.shape)
print(df.describe())
df.head()

In [ ]:
# First things first, handling the notes:

# "You need to define "winter" by yourself."
# Let's define winter. An adequate separator between winters would be
# a summer month that has never had snow over the years. Let's find one.

# Get maximum values for data when grouped by month (regardless of year or day)
df.groupby('Month').agg('max')

# June, July and August (6-8) have always been snowless. So, we can assume
# that a winter lasts from September to May. 

In [ ]:
# Creating a winter-identifier for each data row
#
# If month < 6, data belongs to previous year's winter. If month >= 6,
# data belongs to current year's winter. Doesn't matter which winter
# the snowless months are included in as they don't affect the sums, 
# I just don't want them to have NaN values as this will be used as index.

df['Winter'] = df.apply(lambda x: (str(x['Year'] - 1) + '-' + str(x['Year'])) 
                  if x['Month'] < 6 
                  else (str(x['Year']) + '-' + str(x['Year'] + 1)), axis='columns')
df.head()

In [ ]:
# "FMI uses -1 as snow depth when "there is absolutely no snow at all". 
#  We don't want to reduce snow sum in that case, so replace -1 with 0."

# Replace values
df['Snow depth (cm)'].replace(-1.0, 0.0, inplace=True)

# Sanity check that snow depth in July was 0 cm
df[df.Month == 7].head()

In [ ]:
# " For missing data, assume that the snow depth has been the same as 
#   during the previous day. (Fill any NaNs with the previous valid value.)"

# Fill NaNs with previous value
df['Snow depth (cm)'].fillna(method='ffill', inplace=True)

# Check that no NaN values were left
print(df.shape)
df['Snow depth (cm)'].describe()

# Shape and count of snow depths are the same, so all data is present.

In [ ]:
# Some more data cleanup: drop data for winters that cannot be considered full.
# This includes winters of 1958-1959 and 2020-2021. 
df = df[~(df['Winter'] == '1958-1959') & ~(df['Winter'] == '2020-2021')]

In [ ]:
# Produce snow_sum DataFrame with snow sum, snow_sum rank (largest = 1),
# number of days with snow, max snow depth of winter

# Helper column: count, used to calculate number of days that had snow
df['count'] = df['Snow depth (cm)'] > 0.0

# Dropping unneeded columns
df = df[['Winter', 'Snow depth (cm)', 'count']]
# Also adding max column that can be aggregated when grouped
df['max'] = df['Snow depth (cm)']
# Renaming Snow depth column to Snow sum for the aggregation
df.rename(columns={"Snow depth (cm)": "Snow sum"}, inplace=True) 
df

In [ ]:
# Group by Winter to get aggregated numbers
df = df.groupby('Winter').agg({'Snow sum': 'sum', 'count': 'sum', 'max': 'max'})
df

In [ ]:
# Minor tweaking: change max and snow sum columns into integers
df['max'] = df['max'].astype(int)
df['Snow sum'] = df['Snow sum'].astype(int)

# Get rankings for different winters: 
# This creates a list of numbers from 1 upwards, and a matching list of Year values sorted by amount of snow from most to least
# .T transposes the dataframe, as otherwise we'd have one column per year and only two rows of data.
winter_ranks = pd.DataFrame([range(1, len(df) + 1), df.sort_values(by='Snow sum', ascending=False).index.tolist()]).T
winter_ranks.columns = ['rank', 'Year']
winter_ranks.set_index('Year', inplace=True)
winter_ranks

In [ ]:
# Add rank information to the original data frame
df = df.join(winter_ranks)

# Reorganize columns to match requested data
df = df[['Snow sum', 'rank', 'count', 'max']]
df

## Problem 5. Statfi data wrangle.
- Here we're trying to make some sense out of the data that we downloaded from the statfi service in problem 5 of the first exercises. If you did it successfully, please use your own data. Otherwise you may use the data (from the first exercises model solution) at `public/data_statfin_kihi_pxt_13zt.px.json`.
- The model solution had the keyword `kihi`, and the final table `statfin_kihi_pxt_13zt.px`. You may want to replace them in the url below.
- First use [this front end](https://pxdata.stat.fi/PxWeb/pxweb/en/StatFin/StatFin__kihi/statfin_kihi_pxt_13zt.px/) to produce a meaningful table, and then try to produce a similar table with pandas and your data.
- The most elegant solution wins!

#### Table produced via StatFi UI

(used the kihi table above)

Information:
* Index (1985=100)
* Real Price Index (1985=100)

Region:
* Whole country
* Greater Helsinki
* Whole country excluding Greater Helsinki

Quarter:
* All available quarters


In [ ]:
# Reproduction of table based on data

import json

# Read in data
# The json file is not something that can be just thrown in a df
# so reading it into a dict with json module itself
with open('data_statfin_kihi_pxt_13zt.px.json', 'r') as f:
    data = json.load(f)
    
data.keys()

In [ ]:
# Order of columns in data fields
columns = [column['text'] for column in data['columns']]
columns

# Creation of a similar table requires columns:
# 'Region', 'Quarter', 'Index (1985=100)' and 'Real Price Index (1985=100)'
# These correspond to indexes 0, 1, 2, 3.

# Inspecting the data shows that each row of data contains a dictionary object
# with 'key' containing column indexes 0 + 1, and values containing the rest.
data['data'][0]

In [ ]:
# The needed data can be grabbed off the data section in the following way:
data['data'][0]['key'] + data['data'][0]['values'][0:2]

In [ ]:
# Creation of a DataFrame from the data within the JSON file
df = pd.DataFrame([row['key'] + row['values'][0:2] for row in data['data']])
df

In [ ]:
# Get some basic info out of the data frame
print(df.shape)
df.describe()

In [ ]:
# At this point, the missing values at the end of the dataframe caused questions.
# The user interface for StatFi contained a "API query for this table" section,
# which contained an additional filter that wasn't accounted for previously: 
'''
{
      "code": "Alue",
      "selection": {
        "filter": "item",
        "values": [
          "01",
          "02",
          "03"
        ]
      }
'''
# So, setting some column names for readability and dropping irrelevant data
df.columns = ['Region', 'Quarter', 'IDX', 'RPI']
df = df[df.Region.isin(['01', '02', '03'])]
df.set_index('Quarter', inplace=True)
df

In [ ]:
# Setting up correct regions:
# There was no manual as to which region is which. But, this can be deduced
# based on the numbers. For 1985Q1, RPI for whole country was 98.2, greater
# Helsinki 93.7 and whole country excl. greater Helsinki 98.8.

# Finding 1985Q1 values in df
df.loc['1985Q1']

In [ ]:
# So, Region info can be replaced as follows:
df = df.replace({'Region': {'01': 'Whole country', '02': 'Greater Helsinki', '03': 'Whole country excluding Greater Helsinki'}})
df

In [ ]:
# Before pivots (though not strictly necessary for just the table),
# changing data types of IDX and RPI to floats
df['RPI'] = df.RPI.astype(float)
df['IDX'] = df.IDX.astype(float)

# Pivot 'Region' data into a column
df = df.pivot(columns='Region', values=['IDX', 'RPI'])

# Rename shortened IDX and RPI into their proper names
df = df.rename(columns={'IDX': 'Index (1985=100)', 'RPI': 'Real Price Index (1985=100)'})
df

In [ ]:
# One minor difference in the case above is that Greater Helsinki and 
# Whole country are in a different order in table above and in the UI.

# But, if there's something to do better than the UI, we can plot this 
# data as a graph. The webpage couldn't, as apparently there was too
# much data to plot. :) 

from matplotlib import pyplot as plt

In [ ]:
# Plot index data
df['Index (1985=100)'].plot(figsize=(10, 6))
plt.xticks(rotation=45)
plt.title('Prices of old single family houses, index (1985=100)')
plt.show()

In [ ]:
# That 1990s dip in house prices gives a pretty good
# indication of how bad the recession was.

In [ ]:
# Plot real price indexes
df['Real Price Index (1985=100)'].plot(figsize=(10, 6))
plt.xticks(rotation=45)
plt.title('Prices of old single family houses, Real Price Index (1985=100)')
plt.show()

In [ ]:
# This graph shows it even better, as it factors everything into
# prices where effects of inflation have been ruled out. No wonder
# it was strictly forbidden to write in the exercise books back in 
# elementary school, and everything was reused. 